# Challenge 2: Sentiment Analysis

In this challenge we will learn sentiment analysis and practice performing sentiment analysis on Twitter tweets. 

## Introduction

Sentiment analysis is to *systematically identify, extract, quantify, and study affective states and subjective information* based on texts ([reference](https://en.wikipedia.org/wiki/Sentiment_analysis)). In simple words, it's to understand whether a person is happy or unhappy in producing the piece of text. Why we (or rather, companies) care about sentiment in texts? It's because by understanding the sentiments in texts, we will be able to know if our customers are happy or unhappy about our products and services. If they are unhappy, the subsequent action is to figure out what have caused the unhappiness and make improvements.

Basic sentiment analysis only understands the *positive* or *negative* (sometimes *neutral* too) polarities of the sentiment. More advanced sentiment analysis will also consider dimensions such as agreement, subjectivity, confidence, irony, and so on. In this challenge we will conduct the basic positive vs negative sentiment analysis based on real Twitter tweets.

NLTK comes with a [sentiment analysis package](https://www.nltk.org/api/nltk.sentiment.html). This package is great for dummies to perform sentiment analysis because it requires only the textual data to make predictions. For example:

```python
>>> from nltk.sentiment.vader import SentimentIntensityAnalyzer
>>> txt = "Ironhack is a Global Tech School ranked num 2 worldwide.   Our mission is to help people transform their careers and join a thriving community of tech professionals that love what they do."
>>> analyzer = SentimentIntensityAnalyzer()
>>> analyzer.polarity_scores(txt)
{'neg': 0.0, 'neu': 0.741, 'pos': 0.259, 'compound': 0.8442}
```

In this challenge, however, you will not use NLTK's sentiment analysis package because in your Machine Learning training in the past 2 weeks you have learned how to make predictions more accurate than that. The [tweets data](https://www.kaggle.com/kazanova/sentiment140) we will be using today are already coded for the positive/negative sentiment. You will be able to use the Naïve Bayes classifier you learned in the lesson to predict the sentiment of tweets based on the labels.

## Conducting Sentiment Analysis

### Loading and Exploring Data

The dataset we'll be using today is located on Kaggle (https://www.kaggle.com/kazanova/sentiment140). Once you have downloaded and imported the dataset, it you will need to define the columns names: df.columns = ['target','id','date','flag','user','text']

*Notes:* 

* The dataset is huuuuge (1.6m tweets). When you develop your data analysis codes, you can sample a subset of the data (e.g. 20k records) so that you will save a lot of time when you test your codes.

In [1]:
import numpy as np
import pandas as pd
import pickle

In [2]:
data = pd.read_csv("archive.zip", names = ["Target", "ID", "Date", "Flag", "User", "Text"], compression = "zip", encoding = "iso-8859-2")
data.head()

,Target,ID,Date,Flag,User,Text
0,0,1467810369,Mon Apr 06 22:19:45 PDT 2009,NO_QUERY,_TheSpecialOne_,"@switchfoot http://twitpic.com/2y1zl - Awww, t..."
1,0,1467810672,Mon Apr 06 22:19:49 PDT 2009,NO_QUERY,scotthamilton,is upset that he can't update his Facebook by ...
2,0,1467810917,Mon Apr 06 22:19:53 PDT 2009,NO_QUERY,mattycus,@Kenichan I dived many times for the ball. Man...
3,0,1467811184,Mon Apr 06 22:19:57 PDT 2009,NO_QUERY,ElleCTF,my whole body feels itchy and like its on fire
4,0,1467811193,Mon Apr 06 22:19:57 PDT 2009,NO_QUERY,Karoli,"@nationwideclass no, it's not behaving at all...."


### Prepare Textual Data for Sentiment Analysis

Now, apply the functions you have written in Challenge 1 to your whole data set. These functions include:

* `clean_up()`

* `tokenize()`

* `stem_and_lemmatize()`

* `remove_stopwords()`

Create a new column called `text_processed` in the dataframe to contain the processed data. At the end, your `text_processed` column should contain lists of word tokens that are cleaned up. Your data should look like below:

![Processed Data](data-cleaning-results.png)

In [3]:
import functions

data_sample = data.sample(20000, random_state = 22)

data_sample["Text"] = data_sample["Text"].apply(lambda x: functions.clean_up(x))
data_sample["TextProcessed"] = data_sample["Text"].apply(lambda x: functions.tokenize(x))
data_sample["TextProcessed"] = data_sample["TextProcessed"].apply(lambda x: functions.stem_and_lemmatize(x))
data_sample["TextProcessed"] = data_sample["TextProcessed"].apply(lambda x: functions.remove_stopwords(x))
data_sample.head()

,Target,ID,Date,Flag,User,Text,TextProcessed
1013875,4,1881370700,Fri May 22 04:27:56 PDT 2009,NO_QUERY,FreshPlastic,followfriday foyboy she had skips on her face ...,"[followfriday, foyboy, skip, face, earlier, th..."
1064419,4,1964672525,Fri May 29 14:36:52 PDT 2009,NO_QUERY,CarebearJK,paulaabdul we are finally starting to see the ...,"[paulaabdul, final, start, see, sun, warm, wea..."
290763,0,1995464536,Mon Jun 01 13:31:56 PDT 2009,NO_QUERY,jessicabalicki,w all my old coworkers going to the wake rip m...,"[w, old, cowork, go, wake, rip, michel, xoxo]"
770144,0,2301890082,Tue Jun 23 16:03:01 PDT 2009,NO_QUERY,dlphntat,yeah bravo tv i so wanna learn more about the ...,"[yeah, bravo, tv, wan, na, learn, real, housew..."
1266526,4,1999668574,Mon Jun 01 20:36:58 PDT 2009,NO_QUERY,stevefroisland,jctrick thanks j i think i m almost getting th...,"[jctrick, thank, j, think, almost, get, hang, ..."


In [4]:
data_sample["TextProcessed"]

1013875    [followfriday, foyboy, skip, face, earlier, th...
1064419    [paulaabdul, final, start, see, sun, warm, wea...
290763         [w, old, cowork, go, wake, rip, michel, xoxo]
770144     [yeah, bravo, tv, wan, na, learn, real, housew...
1266526    [jctrick, thank, j, think, almost, get, hang, ...
                                 ...                        
975627     [mlhakim, hey, give, jiayueh, regard, elabor, ...
1107711          [marriag, marvel, congratul, salley, trent]
233340     [love, late, night, ichat, date, adamgrahek, g...
1181513       [get, readi, play, littl, kiddi, today, yayyy]
199409           [mattyjon, know, jealou, sunbath, im, work]
Name: TextProcessed, Length: 20000, dtype: object

### Creating Bag of Words

The purpose of this step is to create a [bag of words](https://en.wikipedia.org/wiki/Bag-of-words_model) from the processed data. The bag of words contains all the unique words in your whole text body (a.k.a. *corpus*) with the number of occurrence of each word. It will allow you to understand which words are the most important features across the whole corpus.

Also, you can imagine you will have a massive set of words. The less important words (i.e. those of very low number of occurrence) do not contribute much to the sentiment. Therefore, you only need to use the most important words to build your feature set in the next step. In our case, we will use the top 5,000 words with the highest frequency to build the features.

In the cell below, combine all the words in `text_processed` and calculate the frequency distribution of all words. A convenient library to calculate the term frequency distribution is NLTK's `FreqDist` class ([documentation](https://www.nltk.org/api/nltk.html#module-nltk.probability)). Then select the top 5,000 words from the frequency distribution.

In [5]:
from nltk import FreqDist

In [6]:
corpus = []

[corpus.append(x) for text in data_sample["TextProcessed"] for x in text]

fdist = FreqDist(corpus)
fdist

FreqDist({'go': 1712, 'get': 1410, 'day': 1326, 'wa': 1320, 'thi': 1217, 'good': 1196, 'work': 1127, 'like': 1066, 'love': 1051, 'got': 955, ...})

In [7]:
# I did this before knowing that the one below existed
# count = list(fdist.values())
# count.sort(reverse = True)
# words = list(fdist)
# corpus_top = [(word, number) for word, number in zip(words, count)]
# corpus_top_5k = corpus_top[0:5000]

In [8]:
# Only need the names
# corpus_5k = fdist.most_common(5000)

In [9]:
corpus_words = list(fdist)[:5000]

In [100]:
filename = "corpus.pkl"
with open(filename, "wb") as file:
    pickle.dump(corpus_words, file)

### Building Features

Now let's build the features. Using the top 5,000 words, create a 2-dimensional matrix to record whether each of those words is contained in each document (tweet). Then you also have an output column to indicate whether the sentiment in each tweet is positive. For example, assuming your bag of words has 5 items (`['one', 'two', 'three', 'four', 'five']`) out of 4 documents (`['A', 'B', 'C', 'D']`), your feature set is essentially:

| Doc | one | two | three | four | five | is_positive |
|---|---|---|---|---|---|---|
| A | True | False | False | True | False | True |
| B | False | False | False | True | True | False |
| C | False | True | False | False | False | True |
| D | True | False | False | False | True | False|

However, because the `nltk.NaiveBayesClassifier.train` class we will use in the next step does not work with Pandas dataframe, the structure of your feature set should be converted to the Python list looking like below:

```python
[
	({
		'one': True,
		'two': False,
		'three': False,
		'four': True,
		'five': False
	}, True),
	({
		'one': False,
		'two': False,
		'three': False,
		'four': True,
		'five': True
	}, False),
	({
		'one': False,
		'two': True,
		'three': False,
		'four': False,
		'five': False
	}, True),
	({
		'one': True,
		'two': False,
		'three': False,
		'four': False,
		'five': True
	}, False)
]
```

To help you in this step, watch the [following video](https://www.youtube.com/watch?v=-vVskDsHcVc) to learn how to build the feature set with Python and NLTK. The source code in this video can be found [here](https://pythonprogramming.net/words-as-features-nltk-tutorial/).

[![Building Features](building-features.jpg)](https://www.youtube.com/watch?v=-vVskDsHcVc)

In [10]:
def find_features(doc):
    
    row = [doc]
    [row.append((item in doc)) for item in corpus_words]
    
    return row

In [19]:
col_names = ["Doc"]
[col_names.append(w) for w in corpus_words]
corpus_df = pd.DataFrame(columns = col_names)
corpus_df

,Doc,go,get,day,wa,thi,good,work,like,love,...,submit,haaaaat,nth,litt,kirk,bloo,intrigu,parlai,chunk,penn


In [20]:
# This was done in 4 batches of 5000 rows in 4 notebooks at the same time

for doc in data_sample["TextProcessed"].iloc[:5000]:
    corpus_df.loc[len(corpus_df.index)] = find_features(doc)
    
corpus_df

,Doc,go,get,day,wa,thi,good,work,like,love,...,submit,haaaaat,nth,litt,kirk,bloo,intrigu,parlai,chunk,penn
0,"[followfriday, foyboy, skip, face, earlier, th...",False,False,False,False,True,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
1,"[paulaabdul, final, start, see, sun, warm, wea...",False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
2,"[w, old, cowork, go, wake, rip, michel, xoxo]",True,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
3,"[yeah, bravo, tv, wan, na, learn, real, housew...",False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
4,"[jctrick, thank, j, think, almost, get, hang, ...",False,True,False,False,True,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4995,[littl],False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
4996,"[dastardlypir, cliffordaz, go, hang]",True,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
4997,"[definit, go, go, store, need, creamer, coffe,...",True,True,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
4998,"[onli, leah, rebecca, come, home, wonder, get,...",False,True,False,False,False,False,True,False,False,...,False,False,False,False,False,False,False,False,False,False


In [21]:
corpus_df.to_csv("matrix1.csv")

In [24]:
matrix1 = pd.read_csv("matrix1.csv", index_col = 0)
matrix2 = pd.read_csv("matrix2.csv", index_col = 0)
matrix3 = pd.read_csv("matrix3.csv", index_col = 0)
matrix4 = pd.read_csv("matrix4.csv", index_col = 0)

In [26]:
matrix_full = pd.concat([matrix1, matrix2, matrix3, matrix4])

,Doc,go,get,day,wa,thi,good,work,like,love,...,submit,haaaaat,nth,litt,kirk,bloo,intrigu,parlai,chunk,penn
0,"['followfriday', 'foyboy', 'skip', 'face', 'ea...",False,False,False,False,True,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
1,"['paulaabdul', 'final', 'start', 'see', 'sun',...",False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
2,"['w', 'old', 'cowork', 'go', 'wake', 'rip', 'm...",True,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
3,"['yeah', 'bravo', 'tv', 'wan', 'na', 'learn', ...",False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
4,"['jctrick', 'thank', 'j', 'think', 'almost', '...",False,True,False,False,True,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4995,"['mlhakim', 'hey', 'give', 'jiayueh', 'regard'...",False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
4996,"['marriag', 'marvel', 'congratul', 'salley', '...",False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
4997,"['love', 'late', 'night', 'ichat', 'date', 'ad...",False,False,False,False,False,True,False,False,True,...,False,False,False,False,False,False,False,False,False,False
4998,"['get', 'readi', 'play', 'littl', 'kiddi', 'to...",False,True,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False


In [27]:
matrix_full["Sentiment"] = [True if x == 4 else False for x in data_sample["Target"]]
matrix_full

,Doc,go,get,day,wa,thi,good,work,like,love,...,haaaaat,nth,litt,kirk,bloo,intrigu,parlai,chunk,penn,Sentiment
0,"['followfriday', 'foyboy', 'skip', 'face', 'ea...",False,False,False,False,True,False,False,False,False,...,False,False,False,False,False,False,False,False,False,True
1,"['paulaabdul', 'final', 'start', 'see', 'sun',...",False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,True
2,"['w', 'old', 'cowork', 'go', 'wake', 'rip', 'm...",True,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
3,"['yeah', 'bravo', 'tv', 'wan', 'na', 'learn', ...",False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
4,"['jctrick', 'thank', 'j', 'think', 'almost', '...",False,True,False,False,True,False,False,False,False,...,False,False,False,False,False,False,False,False,False,True
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4995,"['mlhakim', 'hey', 'give', 'jiayueh', 'regard'...",False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,True
4996,"['marriag', 'marvel', 'congratul', 'salley', '...",False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,True
4997,"['love', 'late', 'night', 'ichat', 'date', 'ad...",False,False,False,False,False,True,False,False,True,...,False,False,False,False,False,False,False,False,False,False
4998,"['get', 'readi', 'play', 'littl', 'kiddi', 'to...",False,True,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,True


In [28]:
matrix_full = matrix_full.reset_index(drop=True)

In [36]:
matrix_full["Sentiment"].value_counts()

False    10110
True      9890
Name: Sentiment, dtype: int64

In [37]:
matrix_feed = matrix_full.drop(columns="Doc")
matrix_feed

,go,get,day,wa,thi,good,work,like,love,got,...,haaaaat,nth,litt,kirk,bloo,intrigu,parlai,chunk,penn,Sentiment
0,False,False,False,False,True,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,True
1,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,True
2,True,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
3,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
4,False,True,False,False,True,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,True
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
19995,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,True
19996,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,True
19997,False,False,False,False,False,True,False,False,True,False,...,False,False,False,False,False,False,False,False,False,False
19998,False,True,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,True


In [85]:
nltk_list = [(dict(matrix_feed.iloc[x,:-1]),matrix_feed["Sentiment"][x]) for x in range(len(matrix_feed))]

### Building and Traininng Naive Bayes Model

In this step you will split your feature set into a training and a test set. Then you will create a Bayes classifier instance using `nltk.NaiveBayesClassifier.train` ([example](https://www.nltk.org/book/ch06.html)) to train with the training dataset.

After training the model, call `classifier.show_most_informative_features()` to inspect the most important features. The output will look like:

```
Most Informative Features
	    snow = True            False : True   =     34.3 : 1.0
	  easter = True            False : True   =     26.2 : 1.0
	 headach = True            False : True   =     20.9 : 1.0
	    argh = True            False : True   =     17.6 : 1.0
	unfortun = True            False : True   =     16.9 : 1.0
	    jona = True             True : False  =     16.2 : 1.0
	     ach = True            False : True   =     14.9 : 1.0
	     sad = True            False : True   =     13.0 : 1.0
	  parent = True            False : True   =     12.9 : 1.0
	  spring = True            False : True   =     12.7 : 1.0
```

The [following video](https://www.youtube.com/watch?v=rISOsUaTrO4) will help you complete this step. The source code in this video can be found [here](https://pythonprogramming.net/naive-bayes-classifier-nltk-tutorial/).

[![Building and Training NB](nb-model-building.jpg)](https://www.youtube.com/watch?v=rISOsUaTrO4)

In [86]:
from nltk import NaiveBayesClassifier

In [88]:
train = nltk_list[:16000]
test = nltk_list[16000:]

In [90]:
nbc = NaiveBayesClassifier.train(train)

In [91]:
nbc.show_most_informative_features()

Most Informative Features
                 headach = True            False : True   =     31.1 : 1.0
                     sad = True            False : True   =     27.3 : 1.0
                    hurt = True            False : True   =     23.0 : 1.0
                  throat = True            False : True   =     16.7 : 1.0
                 stomach = True            False : True   =     14.7 : 1.0
                  inspir = True             True : False  =     14.6 : 1.0
                  welcom = True             True : False  =     11.8 : 1.0
                    lone = True            False : True   =     11.6 : 1.0
                     www = True             True : False  =     10.9 : 1.0
                     cri = True            False : True   =     10.8 : 1.0


### Testing Naive Bayes Model

Now we'll test our classifier with the test dataset. This is done by calling `nltk.classify.accuracy(classifier, test)`.

As mentioned in one of the tutorial videos, a Naive Bayes model is considered OK if your accuracy score is over 0.6. If your accuracy score is over 0.7, you've done a great job!

In [92]:
from nltk.classify import accuracy

In [93]:
accuracy(nbc, test)

0.7395

In [ ]:
# WOOOOOOOOOOOOO!

In [96]:
filename = "nbc.pkl"
with open(filename, "wb") as file:
    pickle.dump(nbc, file)

## Bonus Question 1: Improve Model Performance

If you are still not exhausted so far and want to dig deeper, try to improve your classifier performance. There are many aspects you can dig into, for example:

* Improve stemming and lemmatization. Inspect your bag of words and the most important features. Are there any words you should furuther remove from analysis? You can append these words to further remove to the stop words list.

* Remember we only used the top 5,000 features to build model? Try using different numbers of top features. The bottom line is to use as few features as you can without compromising your model performance. The fewer features you select into your model, the faster your model is trained. Then you can use a larger sample size to improve your model accuracy score.

This lab is very time consuming and the model is already over 0.7 accuracy, thus I will not attempt to improve it.

The steps that I would do with more time would be to reduce the features without compromising accuracy, 5k makes it very slow to build the appropiate structure.

## Bonus Question 2: Machine Learning Pipeline

In a new Jupyter Notebook, combine all your codes into a function (or a class). Your new function will execute the complete machine learning pipeline job by receiving the dataset location and output the classifier. This will allow you to use your function to predict the sentiment of any tweet in real time. 

Done at ```Bonus-Question-2-Solution.ipynb```

## Bonus Question 3: Apache Spark

If you have completed the Apache Spark advanced topic lab, what you can do is to migrate your pipeline from local to a Databricks Notebook. Share your notebook with your instructor and classmates to show off your achievements!

Maybe later!